In [1]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers # requires transformers==4.35.2
device = torch.device('cuda:0')

In [2]:
print(transformers.__version__)

4.43.3


In [3]:
# draft_model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
# draft_model_name ="codellama/CodeLlama-7b-hf"
# draft_model_name = "bigcode/starcoderbase-1b"
draft_model_name = "Qwen/Qwen1.5-0.5B-Chat"
# draft_model_name = "facebook/opt-125m"

draft_model = AutoModelForCausalLM.from_pretrained(draft_model_name, trust_remote_code=True, torch_dtype=torch.float16, use_flash_attention_2=True).to(device)#, load_in_4bit=True)
print(draft_model.device)

The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


cuda:0


In [4]:
# model_name = "deepseek-ai/deepseek-coder-6.7b-instruct"
# model_name="codellama/CodeLlama-70b-hf"
# model_name = "bigcode/starcoderbase"
model_name = "Qwen/Qwen1.5-7B-Chat"
# model_name = "facebook/opt-6.7b"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.float16, use_flash_attention_2=True).to(device)#, load_in_4bit=True)#  , use_flash_attention=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceH4/mt_bench_prompts", split="train")

Generating train split:   0%|          | 0/80 [00:00<?, ? examples/s]

In [6]:
import difflib

@torch.no_grad()
def find_candidate_pred_tokens(input_ids, max_ngram_size=3, num_pred_tokens=10):
    input_length = input_ids.size(1)

    # Ensure max_ngram_size and num_pred_tokens are valid
    if max_ngram_size <= 0 or num_pred_tokens <= 0 or max_ngram_size > input_length:
        raise ValueError("Invalid max_ngram_size or num_pred_tokens")

    for ngram_size in range(max_ngram_size, 0, -1):
        # Extract the last n tokens as our search ngram
        ngram = input_ids[0, -ngram_size:].tolist()

        # Create sliding windows of size ngram_size
        windows = input_ids.unfold(dimension=1, size=ngram_size, step=1)

        # Convert ngram to a tensor for comparison
        ngram_tensor = torch.tensor(ngram, device=input_ids.device).unsqueeze(0)

        # Find where the windows match the ngram
        matches = (windows == ngram_tensor).all(dim=2)

        # Get the indices of matches
        match_indices = matches.nonzero(as_tuple=True)[1]

        # Iterate through match indices to find a valid continuation
        for idx in match_indices:
            start_idx = idx + ngram_size
            end_idx = start_idx + num_pred_tokens
            # Ensure we don't go beyond the length of input_ids and avoid self-match
            # if end_idx <= input_length and start_idx < input_length - ngram_size:
            #     return input_ids[0, start_idx:end_idx]
            if start_idx < input_length - ngram_size:
                return input_ids[0, start_idx:min(end_idx, input_length)]

    # If no match is found, return an empty tensor
    return torch.tensor([100], dtype=torch.long, device=input_ids.device)

In [7]:
from transformers.generation.candidate_generator import CandidateGenerator, _crop_past_key_values
from transformers.generation.stopping_criteria import StoppingCriteria
from transformers.generation.configuration_utils import GenerationConfig
from typing import Tuple, Optional
import time

class PromptLookupCandidateGenerator(CandidateGenerator):
    def __init__(self, input_ids, ngram_size=3, num_pred_tokens=10):
        self.orig_input_len = input_ids.shape[-1]
        self.ngram_size = ngram_size
        self.num_pred_tokens = num_pred_tokens
        self.last_predicted = 0
    
    def get_candidates(self, input_ids: torch.LongTensor) -> Tuple[torch.LongTensor, Optional[torch.FloatTensor]]:
        # print("Getting candidates")
        new_tokens = find_candidate_pred_tokens(input_ids, self.ngram_size, self.num_pred_tokens).unsqueeze(0)
        self.last_predicted = new_tokens.shape[-1]
        
        return torch.cat(
            (
                input_ids,
                new_tokens
            ),
            dim=-1
        ), None
    
    def update_candidate_strategy(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, num_matches: int): # Maybe use the number of matches/scores to have a threshold
        pass
        # if num_matches == self.last_predicted:
        #     self.num_pred_tokens *= 1.5
        # else:
        #     self.num_pred_tokens /= 1.5
        # self.num_pred_tokens = int(self.num_pred_tokens)
        # self.num_pred_tokens = min(self.num_pred_tokens, 100)
        # self.num_pred_tokens = max(self.num_pred_tokens, 1)

class NumRunsStoppingCriteria(StoppingCriteria):
    def __init__(self, max_num_runs=4):
        self.max_num_runs = 4
        self.num_runs = 0

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> torch.BoolTensor:
        self.num_runs += 1
        return self.num_runs >= self.max_num_runs

class ScoreStoppingCriteria:
    def __init__(self, min_score):
        self.min_score = min_score

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> torch.BoolTensor:
        if not(scores):
            # print("No scores")
            return False
        else:
            ...
            # print("Got scores scores stopping: ", scores[0].shape, len(scores))
        scores_tensor = torch.stack(scores, dim=0)
        softmax_scores = F.softmax(scores_tensor, 2)
        # print(softmax_scores)
        return (softmax_scores.max(dim=2).values < self.min_score).any().item()


def _get_default_candidate_generator_generator(generator: CandidateGenerator):
    def _get_candidate_generator(self, **kwargs):
        return generator
    return _get_candidate_generator

class TwoLayerLookupCandidateGenerator(CandidateGenerator):
    def __init__(self, tokenizer, prompt_tokens, draft_model, input_ids, use_score_check=False, min_score=0, scores_count=0, num_runs=4, **diff_prompt_args):
        self.tokenizer = tokenizer
        self.prompt_tokens = prompt_tokens
        self.draft_model = draft_model
        self.input_ids = input_ids
        self.candidate_generator = PromptLookupCandidateGenerator(
            self.input_ids, 
            **diff_prompt_args
        )
        self.draft_model.generation_config.pad_token_id = tokenizer.pad_token_id
        
        self.past_key_values = None
        self.num_runs = num_runs

        self.draft_model._get_candidate_generator = (_get_default_candidate_generator_generator(self.candidate_generator)).__get__(self.draft_model, type(self.draft_model))

        self.start_token_index = self.input_ids.shape[-1]
        self.min_score = min_score
        self.scores_count = scores_count

        self.use_score_check = use_score_check
    
    def get_candidates(self, input_ids: torch.LongTensor) -> Tuple[torch.LongTensor, Optional[torch.FloatTensor]]:
        if self.past_key_values:
            self.past_key_values = _crop_past_key_values(self.draft_model, self.past_key_values, input_ids.shape[-1] - 1)

        stopping_criteria = [NumRunsStoppingCriteria(self.num_runs), 
                            ]
        if self.use_score_check:
            stopping_criteria = [NumRunsStoppingCriteria(self.num_runs), 
                                 ScoreStoppingCriteria(self.min_score)
                                ]

        # if self.past_key_values:
        #     print(self.past_key_values[0][0].shape)

        if self.past_key_values: 
            generation = self.draft_model.generate(
                inputs=input_ids,
                attention_mask=torch.ones(input_ids.shape[-1], device=input_ids.device).unsqueeze(0),
                prompt_lookup_num_tokens=1,
                max_new_tokens=1000,
                stopping_criteria=stopping_criteria,
                past_key_values=self.past_key_values,
                use_cache=True,
                # output_logits=True,
                output_scores=True,
                return_dict_in_generate=True
            )
        else:
            generation = self.draft_model.generate(
                inputs=input_ids,
                attention_mask=torch.ones(input_ids.shape[-1], device=input_ids.device).unsqueeze(0),
                prompt_lookup_num_tokens=1,
                max_new_tokens=1000,
                stopping_criteria=stopping_criteria,
                use_cache=True,
                # output_logits=True,
                output_scores=True,
                return_dict_in_generate=True
            )
        # print("Scores: ", generation.scores)

        self.pred_tokens_count = generation.sequences.shape[-1] - input_ids.shape[-1]
        self.past_key_values = generation.past_key_values
        self.past_top_scores = torch.stack(generation.scores, dim=1).max(dim=1).values[0]

        return generation.sequences, torch.stack(generation.scores, dim=1)

    def update_candidate_strategy(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, num_matches: int):
        if num_matches == self.pred_tokens_count:
            if self.scores_count == 0:
                self.min_score = 0
            else:
                self.min_score = (self.scores_count / self.scores_count + 1) * (self.min_score)
        else:
            if self.scores_count == 0:
                self.min_score = self.past_top_scores[-num_matches]
            else:
                self.min_score = (self.scores_count / (self.scores_count + 1)) * (self.min_score) + (1 / (self.scores_count + 1)) * (self.past_top_scores[-1])
        self.scores_count += 1
        pass 

In [8]:
def print_update(dictionary):
    for key in dictionary:
        print("\t", key, ": ", dictionary[key][-1])
    print("======")

In [11]:
from tqdm import tqdm
from transformers import TextStreamer, StopStringCriteria
from rapidfuzz.distance import Levenshtein

# lookup_tokens = [10, 20, 40, 60]
# lookup_tokens = [20, 40, 80, 120]
# lookup_tokens = [100, 120]
lookup_tokens = [5, 10, 20]
stats = {lt: {"method": [], "method_with_score_cutoff": [0], "assisted": [], "pld": [], "regular": [], "lev_similarity": [0], "generated_tokens": [0]} for lt in lookup_tokens}

global_min_score = 0
global_scores_count = 0

regular_get_candidate_generator = model._get_candidate_generator

# for row in tqdm(ds):

#     # Regular
#     prev_messages = []
#     start_time = time.perf_counter()
    
#     for message in row['prompt']:
#         prev_messages += [{
#             "role": "user",
#             "content": message
#         }]
#         inputs = tokenizer.apply_chat_template(prev_messages, return_tensors="pt", add_generation_prompt=True, tokenize=True).to(model.device)
#         starting_input_tokens = inputs.shape[-1]
#         max_new_tokens = 500
#         regular_outputs = model.generate(
#             input_ids=inputs,
#             max_new_tokens=max_new_tokens,
#             return_dict_in_generate=True,
#             output_scores=True,
#             # streamer=TextStreamer(tokenizer)
#             # stopping_criteria=[StopStringCriteria(tokenizer, ["\n"])]
#         )
#         new_text = tokenizer.batch_decode(regular_outputs.sequences[:, starting_input_tokens:])[0]
#         prev_messages += [{
#             "role": "assistant",
#             "content": new_text
#         }]
        
#     end_time = time.perf_counter()
#     for lt in lookup_tokens:
#         stats[lt]["regular"] = end_time - start_time

#     print("Regular: ", end_time - start_time)

#     # Assisted
#     prev_messages = []
#     start_time = time.perf_counter()
    
#     for message in row['prompt']:
#         prev_messages += [{
#             "role": "user",
#             "content": message
#         }]
#         inputs = tokenizer.apply_chat_template(prev_messages, return_tensors="pt", add_generation_prompt=True, tokenize=True).to(model.device)
#         starting_input_tokens = inputs.shape[-1]
#         max_new_tokens = 500
#         assisted_outputs = model.generate(
#             input_ids=inputs,
#             max_new_tokens=max_new_tokens,
#             return_dict_in_generate=True,
#             output_scores=True,
#             assistant_model=draft_model
#             # streamer=TextStreamer(tokenizer)
#             # stopping_criteria=[StopStringCriteria(tokenizer, ["\n"])]
#         )
#         new_text = tokenizer.batch_decode(assisted_outputs.sequences[:, starting_input_tokens:])[0]
#         prev_messages += [{
#             "role": "assistant",
#             "content": new_text
#         }]
        
#     end_time = time.perf_counter()
#     for lt in lookup_tokens:
#         stats[lt]["assisted"] = end_time - start_time

#     print("Assisted: ", end_time - start_time)

for lt in lookup_tokens:
    for row in tqdm(ds):
        # input_text = f"Article: {row['article']} Summary: "
        # inputs = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

        # PLD
        prev_messages = []
        start_time = time.perf_counter()

        model._get_candidate_generator = (regular_get_candidate_generator).__get__(model, type(model))
        
        for message in row['prompt']:
            prev_messages += [{
                "role": "user",
                "content": message
            }]
            inputs = tokenizer.apply_chat_template(prev_messages, return_tensors="pt", add_generation_prompt=True, tokenize=True).to(model.device)
            starting_input_tokens = inputs.shape[-1]
            max_new_tokens = 500
            pld_outputs = model.generate(
                input_ids=inputs,
                max_new_tokens=max_new_tokens,
                return_dict_in_generate=True,
                output_scores=True,
                prompt_lookup_num_tokens=lt
                # streamer=TextStreamer(tokenizer)
                # stopping_criteria=[StopStringCriteria(tokenizer, ["\n"])]
            )
            new_text = tokenizer.batch_decode(pld_outputs.sequences[:, starting_input_tokens:])[0]
            prev_messages += [{
                "role": "assistant",
                "content": new_text
            }]
            
        end_time = time.perf_counter()
        stats[lt]["pld"] = end_time - start_time

        print("PLD: ", end_time - start_time)

        # Two Layer
        prev_answers = []
        start_time = time.perf_counter()
        two_layer_candidate_generator = TwoLayerLookupCandidateGenerator(
            tokenizer,
            inputs.shape[-1],
            draft_model,
            inputs,
            use_score_check=False,
            min_score=global_min_score,
            scores_count=global_scores_count,
            ngram_size=5,
            num_pred_tokens=lt,
        )
        model._get_candidate_generator = (_get_default_candidate_generator_generator(two_layer_candidate_generator)).__get__(model, type(model))
        for message in row['prompt']:
            prev_messages += [{
                "role": "user",
                "content": message
            }]
            inputs = tokenizer.apply_chat_template(prev_messages, return_tensors="pt", add_generation_prompt=True, tokenize=True).to(model.device)
            starting_input_tokens = inputs.shape[-1]
            max_new_tokens = 500
            pld_outputs = model.generate(
                input_ids=inputs,
                max_new_tokens=max_new_tokens,
                return_dict_in_generate=True,
                output_scores=True,
                prompt_lookup_num_tokens=lt
                # streamer=TextStreamer(tokenizer)
                # stopping_criteria=[StopStringCriteria(tokenizer, ["\n"])]
            )
            new_text = tokenizer.batch_decode(pld_outputs.sequences[:, starting_input_tokens:])[0]
            prev_messages += [{
                "role": "assistant",
                "content": new_text
            }]
        end_time = time.perf_counter()
        stats[lt]["method"] = end_time - start_time

        print("Two layer: ", end_time - start_time)
print(stats)

  0%|                                                                                                                                                     | 0/80 [00:00<?, ?it/s]

PLD:  21.20741204544902


  1%|█▋                                                                                                                                         | 1/80 [00:54<1:12:12, 54.84s/it]

Two layer:  33.62809154391289
PLD:  15.661539539694786


  2%|███▌                                                                                                                                         | 2/80 [01:31<57:36, 44.31s/it]

...

Two layer:  18.371670972555876
PLD:  8.95885856822133


 15%|█████████████████████                                                                                                                       | 12/80 [06:33<28:54, 25.51s/it]

Two layer:  22.361585844308138
PLD:  27.28148141875863


 15%|█████████████████████                                                                                                                       | 12/80 [07:21<41:41, 36.79s/it]


KeyboardInterrupt: 

In [12]:
import json
stats_file = open("stats_qwen_mtbench.json", "w+")
stats_file.write(json.dumps(stats))
stats_file.close()